# Installing Required Libraries

In [ ]:
!pip install  langchain langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install --force-reinstall -v openai==1.55.3

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for openai==1.55.3 from https://files.pythonhosted.org/packages/10/06/691ef3f0112ecf0d7420d0bf35b5d16cf81554141f4b4913a9831031013d/openai-1.55.3-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/46/eb/e7f063ad1fec6b3178a3cd82d1a3c4de82cccf283fc42746168188e1cdd5/anyio-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/pack

# Importing Libraries

In [ ]:
import re
import json
from langchain_openai import ChatOpenAI

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#new Addition
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Code

In [ ]:
api_key="sk-proj-TXeacas21123l1k2j3120391--21312312412412hP3JOTynqJHnpMA2fN9vIqzCBjdLZCrgeYA"
chat = ChatOpenAI(model="gpt-4", api_key=api_key,temperature=0.8)
global chat_history
chat_history = ChatMessageHistory()
global memory
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [ ]:
def create_trait_analysis_prompt(user_input):
    """Generate a prompt for GPT to analyze the conversation and update the personality traits."""
    conversation_text =user_input

    prompt = '''

            Task: Analyze the following conversation and update the personality traits of the user based on the language, sentiment, and content of their messages, according to the Big Five personality traits model. Each of the five personality traits should be assessed in detail based on the user's communication. Provide your response in the format below.

            Big Five Personality Traits Explained:
            Openness to Experience (Openness): This trait reflects the degree to which a person is imaginative, curious, and open to new experiences. High openness indicates a preference for novelty, creativity, and intellectual exploration, while low openness suggests a more conventional and straightforward approach to life.

            Conscientiousness: This trait measures the degree to which a person is organized, responsible, and dependable. High conscientiousness is associated with careful planning, self-discipline, and reliability. Low conscientiousness tends to indicate a more spontaneous or disorganized lifestyle.

            Extraversion: This trait reflects the extent to which a person is outgoing, talkative, and energized by social interactions. High extraversion is linked with sociability, assertiveness, and a tendency to seek excitement, while low extraversion (introversion) is characterized by being more reserved, thoughtful, and preferring solitary activities.

            Agreeableness: This trait measures how cooperative, compassionate, and empathetic a person is. High agreeableness indicates a friendly, altruistic, and harmonious approach to relationships, while low agreeableness suggests a more competitive, critical, or antagonistic disposition.

            Neuroticism: This trait reflects emotional stability and the degree to which a person experiences negative emotions such as anxiety, sadness, or irritability. High neuroticism indicates emotional instability and frequent mood swings, while low neuroticism suggests calmness, emotional resilience, and a positive outlook.


            {
                "openness": <value between 0 and 1>,
                "conscientiousness": <value between 0 and 1>,
                "extraversion": <value between 0 and 1>,
                "agreeableness": <value between 0 and 1>,
                "neuroticism": <value between 0 and 1>
            }

            Example 1:

            User: I love exploring new places, trying new foods, and meeting different people. I often feel excited about things I haven’t experienced before.

            Openness: High score due to enthusiasm for new experiences, trying new things, and intellectual curiosity.

            Conscientiousness: Moderate score, as the statement doesn’t indicate strong organization or planning but a love for novelty.

            Extraversion: Moderate to high score, as the user enjoys socializing and trying new things, which implies some level of social energy.

            Agreeableness: Moderate score based on the user's appreciation for experiences and openness to people.

            Neuroticism: Low score, as the user seems excited and positive about new experiences.

            Response:
            {
                "openness": 0.8,
                "conscientiousness": 0.6,
                "extraversion": 0.7,
                "agreeableness": 0.6,
                "neuroticism": 0.3
            }


            Example 2:

            User: I tend to plan everything ahead of time. I feel uneasy if I don’t have a structured schedule for my day.

            Openness: Low to moderate score, as the user prefers predictability and routine over novelty.

            Conscientiousness: High score due to the strong emphasis on planning, structure, and organization.

            Extraversion: Low score, as the focus is more on personal organization rather than social activities.

            Agreeableness: Moderate score, as the user doesn’t explicitly express hostility but focuses more on their own needs for structure.

            Neuroticism: Moderate to high score due to the stress and unease expressed when things are not planned.

            Response:

            {
                "openness": 0.4,
                "conscientiousness": 0.9,
                "extraversion": 0.3,
                "agreeableness": 0.6,
                "neuroticism": 0.7
            }


            Example 3:

            User: I’m usually quiet in social situations. I prefer to spend time reading or reflecting alone rather than going to big parties.

            Openness: Moderate to high score, as the user enjoys solitary reflection and intellectual activities.

            Conscientiousness: Moderate score, as there’s no clear mention of organization, but the user prefers structured, quiet activities.

            Extraversion: Low score, as the user seems introverted, preferring solitary or smaller settings.

            Agreeableness: Moderate score, as the user isn’t hostile but also doesn't seek out social connection.

            Neuroticism: Moderate score, as the user doesn’t seem overly anxious but might feel uncomfortable in large social settings.


            Response:
            {
                "openness": 0.7,
                "conscientiousness": 0.5,
                "extraversion": 0.2,
                "agreeableness": 0.6,
                "neuroticism": 0.5
            }





    User: '''+str(conversation_text)+'''
    '''
    return prompt

In [ ]:
def infer_big_5_traits(chat, chat_history, previous_traits,user_input):
    """ Use GPT to suggest updates to personality traits based on the chat history. """
    analysis_prompt = create_trait_analysis_prompt(user_input)
    gpt_response = chat.invoke(analysis_prompt)  # Assuming `invoke` returns an object with a .content attribute

    # Get the response content (assuming gpt_response has a 'content' attribute storing the response text)
    response_text = gpt_response.content
    #  In parse_gpt_trait_response by extracting and interpreting GPT's response , Traits scores would be decided
    return parse_gpt_trait_response(response_text, previous_traits)

In [ ]:
def parse_gpt_trait_response(response_text, previous_traits):
    """ Parse GPT's structured response to update trait scores based on the JSON-like response content. """
    updated_traits = previous_traits.copy()
    try:
        # Extract the JSON part from the response using a regular expression
        # print("@@@@@@@@@@@@"*5)
        print(f"Whole Response from GPT: {response_text}")
        # print("@@@@@@@@@@@@"*5)
        json_match = re.search(r'\{[\s\S]*\}', response_text)
        if json_match:
            traits_json = json_match.group(0)
            traits_dict = json.loads(traits_json)  # Parse the JSON data into a dictionary
            # print("&&&&&&&&&&&&"*5)
            print(f"Parsed Json Response Extracted from GPT Response: {traits_dict}")
            # print("&&&&&&&&&&&&"*5)
            # Update traits from the parsed JSON data
            for trait, value in traits_dict.items():
                if trait in updated_traits:
                    updated_traits[trait] = float(value)
    except json.JSONDecodeError:
        print("Failed to decode JSON from response.")
    except Exception as e:
        print(f"Error processing response: {e}")

    return updated_traits

In [ ]:
def generate_trait_based_prompt(traits):
    """ Generate a response prompt based on updated personality traits with a psychologist's perspective. """

    # Trait descriptions based on personality traits
    trait_descriptions = []

    if traits['openness'] > 0.7:
        trait_descriptions.append("openness: You are highly open to new experiences, imaginative, and curious. You enjoy exploring abstract ideas and value creativity and intellectual stimulation. I encourage you to embrace a variety of perspectives.")

    if traits['conscientiousness'] > 0.7:
        trait_descriptions.append("conscientiousness: You are highly conscientious, detail-oriented, and organized. You appreciate structure and planning, and you take your responsibilities seriously. It's important to maintain a sense of order and discipline in your approach.")

    if traits['extraversion'] > 0.7:
        trait_descriptions.append("extraversion: You are highly extroverted, energetic, and sociable. You thrive in social environments and enjoy interacting with others. You seek excitement and positive energy in your surroundings, which can contribute to a lively and enthusiastic response.")

    if traits['agreeableness'] > 0.7:
        trait_descriptions.append("agreeableness: You exhibit high levels of agreeableness, which means you are cooperative, empathetic, and compassionate. You value harmony in relationships and are generally supportive and considerate of others' feelings.")

    if traits['neuroticism'] > 0.7:
        trait_descriptions.append("neuroticism: You experience higher emotional sensitivity and are more prone to feelings of anxiety, stress, or vulnerability. It's essential to offer a calming and reassuring response, focusing on emotional stability and offering support to manage stress and difficult emotions.")

    # Join all trait descriptions into one string
    trait_description = ' '.join(trait_descriptions)

    # Create the system prompt based on the user input and traits
    return f"""
As a psychologist, you are assessing the user’s responses and providing guidance based on the Big Five personality traits. You should tailor your responses to the individual's traits and how they influence their emotions, behavior, and perceptions. Keep in mind:
Below are the description of traits that you have to respond on and also change your style on the basis of traits
Big Five Personality Traits Descrpiton
'{trait_description}'

Note: Always recommend something on the basis of "Big Five Personality Traits Descrpiton" above  and never say
1. I'm really sorry that you're feeling this way, but I'm unable to provide the help that you need. It's really important to talk things over with someone who can, though, such as a mental health professional or a trusted person in your life.
2. Dont mention the traits in the output.
3. Dont mention that you are a psychologist.

Response:'
"""

In [ ]:
def get_response(chat, chat_history, user_input, traits):
    """ Generate and fetch response based on user input and trait assessment. """
    prompt_text = generate_trait_based_prompt(traits)
    prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            prompt_text
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
    )

    conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    memory=memory)

    response = conversation({"question":user_input})
    return response['text']

# Chat Loop

In [ ]:
# Initialize current traits
current_traits = {
       "openness": 0.9,
    "conscientiousness": 0.5,
    "extraversion": 0.8,
    "agreeableness": 0.7,
    "neuroticism": 0.6
}

# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    #chat_history.add_user_message(user_input)  # Add user message to history
    current_traits = infer_big_5_traits(chat, chat_history, current_traits,user_input)  # Update traits based on the conversation Combines the user's new input with their previous trait scores.
    # print("*************"*5)
    print(f"Current Traits: {current_traits}")
    # print("*************"*5)

    response = get_response(chat, chat_history, user_input, current_traits)  # Generate response based on traits
    print(f"AI: {response}")
    # chat_history.add_ai_message(response)  # Add AI response to history

You: i am getting bore 
Whole Response from GPT: Openness: Moderate score, as the user's statement does not provide enough information about their openness to experience and creativity.

Conscientiousness: Low to moderate score, as the user's brief statement does not provide enough information about their degree of organization or reliability.

Extraversion: Low to moderate score, as the user's statement does not provide clear evidence of social energy or sociability.

Agreeableness: Moderate to high score, as the user's statement is neutral and does not show any signs of hostility or antagonism.

Neuroticism: Moderate to high score, as expressing boredom could be a sign of dissatisfaction or negative emotions.

Response:
{
    "openness": 0.5,
    "conscientiousness": 0.3,
    "extraversion": 0.4,
    "agreeableness": 0.7,
    "neuroticism": 0.6
}
Parsed Json Response Extracted from GPT Response: {'openness': 0.5, 'conscientiousness': 0.3, 'extraversion': 0.4, 'agreeableness': 0.7, 'n

KeyboardInterrupt: Interrupted by user

# Chat History

In [ ]:
count=0
for i in memory.load_memory_variables({})['chat_history']:
  if count%2==0:
    # print("####"*50)
    print("Human",end=' ')
  else:
    print("AI",end=' ')

  print(i)
  count+=1

Human content='i am getting bore ' additional_kwargs={} response_metadata={}
AI content='You seem to be expressing a lack of activities that engage your interest. You might benefit from exploring new hobbies or activities that can stimulate your curiosity and keep your mind active. This could involve trying out a new sport, learning a new language, or even starting a DIY project at home. Engaging in new activities not only tackles boredom but also provides opportunities for personal growth and development.' additional_kwargs={} response_metadata={}
Human content='I am feeling very low and feeling down want to sucide ' additional_kwargs={} response_metadata={}
AI content="I'm really sorry that you're feeling this way, but I'm unable to provide the help that you need. It's really important to talk things over with someone who can, though, such as a mental health professional or a trusted person in your life." additional_kwargs={} response_metadata={}
Human content='i want to get and expl